# Ten thousand tweets as @BrainStraining

Today, I reached my 9,999 using the handle [@BrainStraining](https://twitter.com/BrainStraining). For my 10,000th tweet I wanted to both celebrate and see how I got here.  Earlier in the year I read a wonderfully geeky blog post by Julia Silge who used RStats to run some [cool analyses on her own first 10,000 tweets](http://juliasilge.com/blog/Ten-Thousand-Tweets/). She share That seemed like a good idea so I am forking it.

Since I doubt I can add much to the good work that Julia did on the analysis side, I thought I'd see if I could add anything on the sharing side. So I thought doing things using a combination of Jupyter notebooks and Github. This is the result. 

Share and enjoy,  
Caspar Addyman  
[@BrainStraining](https://twitter.com/BrainStraining)

In [5]:
library(ggplot2)
library(lubridate)
library(scales)


Warning message:
: package 'lubridate' was built under R version 3.2.3